In [ ]:
import json
from typing import re
import pandas as pd
import re
!pip install pygooglenews
from pygooglenews import GoogleNews
import requests
from bs4 import BeautifulSoup
from googlesearch import search
!pip install flair
!pip install sentence-transformers
from flair.embeddings import SentenceTransformerDocumentEmbeddings
from flair.data import Sentence
from scipy.spatial.distance import cosine, jaccard, cdist
from google.colab import drive
!pip install advertools
import advertools as adv
import gzip
import datetime as dt
from nltk import ngrams
import torch
import tensorflow as tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 26.4 MB/s 
     |████████████████████████████████| 1.2 MB 41.5 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 128 kB 58.4 MB/s 
  Created wheel for feedparser: filename=feedparser-5.2.1-py3-none-any.whl size=44952 sha256=7a10154142af633399fcccd15aa61173a42abf77f2345998f899bae974e6a2f3
  Stored in directory: /root/.cache/pip/wheels/29/bf/46/b4a597d435d3aee6c2fa583824897336d65abf13ebe3405b70
Successfully built feedparser
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
ERROR: pip's dependency resol

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import the Clusters Data

In [ ]:
tweets_df = pd.read_pickle("/content/drive/MyDrive/FinalTweetsProjectData/25-4-22.pkl") # selected_clusters = (39,38,53,35,33,19)
df_filtered = tweets_df[tweets_df['topic'] == 39] # 39,38,53
df_filtered
# indexes = tweets_df.index[tweets_df['topic'] in selected_clusters]
# indexes
# check.drop(check[check.topic not in selected_clusters].index, inplace=True)

,topic,text,id
84,39,״ממני תראו וכך תעשו״.\nלא אכפת ולא חשוב מה עשו...,1518334092659544068
294,39,נתניהו: לא הוגן להשוות את הוצאות המזון לפי גוד...,1518334549775769600
407,39,"@DaphnaLiel חוקי, כן. עלות סבירה וזולה משמעותי...",1518334832874561536
414,39,@idanlib1 @sharongal צודק. תיקי נתניהו אכן הרע...,1518334860292771840
467,39,"כלומר, בניגוד להודעת בנט, הוצאות הטייק-אווי הפ...",1518334948222160896
...,...,...,...
121041,39,בו כזבית שכר ציוד וולט כדי להעביר את 50000 השק...,1518685689856106497
121622,39,"@N12News @K82GiF4oGDNFgga @yaronavraham האמת ,...",1518686812314779648
123490,39,קורא לבנט לקחת את שכרו כראש ממשלה ולממן ממנו א...,1518690749541498885
124452,39,@NoaLavi6 @IzralevichMeir @GalitDistel אגב הגב...,1518692955850260480


In [ ]:
def print_top_n_Ngrams(scores_dict, n):
    sorted_terms = sorted(scores_dict, key=scores_dict.get, reverse=True)[:n]
    for term in sorted_terms:
      print("headline: ", term, "score: ", scores_dict[term])



DATA PREP

In [ ]:
class datapre:
    re_to_re=['\'','\"','_',"-",',']
    re_to_remove = ['http\S+', '\n', '#\S+', '@\S+','\r','[A-Z]+','[a-z]+']#, '#\S+'
    stop_word_heb = [' אבל ', ' או ', ' אולי ', ' אותה ', ' אותו ', ' אותי ', ' אותך ', ' אותם ', ' אותן ', ' אותנו ', ' אז ', ' אחר ', ' אחרות ', ' אחרי ', ' אחריכן ', ' אחרים ', ' אחרת ', ' אי ', ' איזה ', ' איך ', ' אין ', ' איפה ', ' איתה ', ' איתו ', ' איתי ', ' איתך ', ' איתכם ', ' איתכן ', ' איתם ', ' איתן ', ' איתנו ', ' אך ', ' אל ', ' אלה ', ' אלו ', ' אם ', ' אנחנו ', ' אני ', ' אס ', ' אף ', ' אצל ', ' אשר ', ' את ', ' אתה ', ' אתכם ', ' אתכן ', ' אתם ', ' אתן ', ' באיזו מידה ', ' באמצע ', ' באמצעות ', ' בגלל ', ' בין ', ' בלי ', ' במידה ', ' במקום שבו ', ' ברם ', ' בשביל ', ' בשעהש ', ' בתוך ', ' גם ', ' דרך ', ' הוא ', ' היא ', ' היה ', ' היכן ', ' היתה ', ' היתי ', ' הם ', ' הן ', ' הנה ', ' הסיבה שבגללה ', ' הרי ', ' ואילו ', ' ואת ', ' זאת ', ' זה ', ' זות ', ' יהיה ', ' יוכל ', ' יוכלו ', ' יותר מדי ', ' יכול ', ' יכולה ', ' יכולות ', ' יכולים ', ' יכל ', ' יכלה ', ' יכלו ', ' יש ', ' כאן ', ' כאשר ', ' כולם ', ' כולן ', ' כזה ', ' כי ', ' כיצד ', ' כך ', ' ככה ', ' כל ', ' כלל ', ' כמו ', ' כן ', ' כפי ', ' כש ', ' לא ', ' לאו ', ' לאיזו תכלית ', ' לאן ', ' לבין ', ' לה ', ' להיות ', ' להם ', ' להן ', ' לו ', ' לי ', ' לכם ', ' לכן ', ' למה ', ' למטה ', ' למעלה ', ' למקום שבו ', ' למרות ', ' לנו ', ' לעבר ', ' לעיכן ', ' לפיכך ', ' לפני ', ' מאד ', ' מאחורי ', ' מאיזו סיבה ', ' מאין ', ' מאיפה ', ' מבלי ', ' מבעד ', ' מדוע ', ' מה ', ' מהיכן ', ' מול ', ' מחוץ ', ' מי ', ' מכאן ', ' מכיוון ', ' מלבד ', ' מן ', ' מנין ', ' מסוגל ', ' מעט ', ' מעטים ', ' מעל ', ' מצד ', ' מקום בו ', ' מתחת ', ' מתי ', ' נגד ', ' נגר ', ' נו ', ' עד ', ' עז ', ' על ', ' עלי ', ' עליה ', ' עליהם ', ' עליהן ', ' עליו ', ' עליך ', ' עליכם ', ' עלינו ', ' עם ', ' עצמה ', ' עצמהם ', ' עצמהן ', ' עצמו ', ' עצמי ', ' עצמם ', ' עצמן ', ' עצמנו ', ' פה ', ' רק ', ' שוב ', ' של ', ' שלה ', ' שלהם ', ' שלהן ', ' שלו ', ' שלי ', ' שלך ', ' שלכה ', ' שלכם ', ' שלכן ', ' שלנו ', ' שם ', ' תהיה ', ' תחת ']
    for w in adv.stopwords['hebrew']:
      stop_word_heb.append(" "+w+" ")
    stop_word_heb=list(set(stop_word_heb))
    re_to_remove_all=re_to_remove+stop_word_heb
    def clean_text(self,text):
        cleantext = re.sub('|'.join(self.re_to_remove_all), ' ', text)
        cleantext = re.sub('|'.join(self.re_to_re), '', cleantext)
        cleantext = re.sub(r'[^\w]', ' ', cleantext)
        cleantext = re.sub(' +', ' ', cleantext)
        return cleantext

indexer

In [ ]:
class indexer:
      def __init__(self, tweets_cluster):
        self.data_prepare= datapre()
        self.ngrams_dict = {}
        self.tweets_cluster = tweets_cluster
        for i, tweet in tweets_cluster.iterrows():
          self.get_n_grams(tweet)

      def get_n_grams(self, tweet):
        # df ,topic , text , tweet_id
          cleantext = self.data_prepare.clean_text(tweet["text"])
          g2=list(ngrams(cleantext.split(), 2))
          g3=list(ngrams(cleantext.split(), 3))
          g4=list(ngrams(cleantext.split(), 4))

          all_ngrams = g2 + g3 + g4
          for gram in all_ngrams:
            if frozenset(gram) not in self.ngrams_dict:
              self.ngrams_dict[frozenset(gram)]=1
            else:
              self.ngrams_dict[frozenset(gram)] +=1

In [ ]:
idx = indexer(df_filtered)
i = 0
for item in idx.ngrams_dict.items():
  print(item)
  if i == 10:
    break
  i+=1

(frozenset({'ממני', 'תראו'}), 1)
(frozenset({'תראו', 'וכך'}), 1)
(frozenset({'תעשו', 'וכך'}), 1)
(frozenset({'לא', 'תעשו'}), 1)
(frozenset({'לא', 'אכפת'}), 1)
(frozenset({'ולא', 'אכפת'}), 1)
(frozenset({'ולא', 'חשוב'}), 1)
(frozenset({'חשוב', 'עשו'}), 1)
(frozenset({'קודם', 'עשו'}), 1)
(frozenset({'קודם', 'נתניהו'}), 1)
(frozenset({'ומשפחתו', 'נתניהו'}), 4)


CleanTitles

In [ ]:
class CleanTitles:
  def __init__(self):
      self.data_prepare= datapre()

  def clean_title(self,full_title):
      full_title = self.data_prepare.clean_text(full_title)
      if "_" in full_title:
        full_title = full_title.replace("_", " ")
      if '"' in full_title:
        full_title = full_title.replace('"', "_")
      full_title = re.sub('\W+', ' ', full_title)
      full_title = full_title.replace("_ ", " ")
      full_title = full_title.replace(" _", " ")
      full_title = full_title.replace("_", '"')
      return full_title

  def generate_ngrams(self, titles, ngrams_min = 2, ngrams_max = 4):
      titles = list(set(titles))
      res = {}
      result = []
      for title in titles:
        for i in range(ngrams_min, ngrams_max, 1):
          curr = list(ngrams(title.split(), i))
          for gram in curr:
            frozen = frozenset(gram)
            if frozen not in res:
              res[frozen] = 1
      for key in res.keys():
        ngram = " ".join(list(key))
        result.append(ngram)

      return result

  def generate__all_kinds_of_ngrams(self, titles, ngrams_min = 2, ngrams_max = 4):
      titles = list(set(titles))
      res = {}
      result = []
      for title in titles:
        for i in range(ngrams_min, ngrams_max, 1):
          curr = list(ngrams(title.split(), i))
          for gram in curr:
            if gram not in res:
              res[gram] = 1
      for key in res.keys():
        ngram = " ".join(list(key))
        result.append(ngram)

      return result

In [ ]:
headline = ["אוקראינה רוסיה חידשה המתקפה הכוחות האוקראיניים במריופול וואלה חדשות", "הלחץ כלכלת ארצות הברית צפוי להימשך בתקופה הקרובה אייס", "חידשה אוקראינה רוסיה"]
clean = CleanTitles()
print(len(clean.generate__all_kinds_of_ngrams(headline)))
print("**********************************************************")
print(len(clean.generate_ngrams(headline)))
clean.generate_ngrams(headline)

32
**********************************************************
31


['כלכלת הלחץ',
 'ארצות כלכלת',
 'ארצות הברית',
 'הברית צפוי',
 'צפוי להימשך',
 'להימשך בתקופה',
 'בתקופה הקרובה',
 'אייס הקרובה',
 'ארצות כלכלת הלחץ',
 'ארצות כלכלת הברית',
 'ארצות הברית צפוי',
 'הברית להימשך צפוי',
 'צפוי להימשך בתקופה',
 'להימשך בתקופה הקרובה',
 'אייס בתקופה הקרובה',
 'רוסיה אוקראינה',
 'חידשה רוסיה',
 'חידשה המתקפה',
 'הכוחות המתקפה',
 'הכוחות האוקראיניים',
 'האוקראיניים במריופול',
 'וואלה במריופול',
 'חדשות וואלה',
 'חידשה רוסיה אוקראינה',
 'חידשה רוסיה המתקפה',
 'חידשה הכוחות המתקפה',
 'הכוחות האוקראיניים המתקפה',
 'הכוחות האוקראיניים במריופול',
 'וואלה האוקראיניים במריופול',
 'וואלה חדשות במריופול',
 'חידשה אוקראינה']

Google APIS

In [ ]:
#  this class search sentences in google and analyze the results and get potential headlines as result
class GoogleModel:
    def __init__(self):
      self.clean_titles= CleanTitles()

    def get_num_result_for_query(self,query, num_to_retrieve, filter_result=False, ):
        title_list = []
        # searching for the results for given query
        res = search(query, tld="co.il", lang='he', num=num_to_retrieve, stop=10, pause=2)
        try:
            for url in res:
                reqs = requests.get(url)
                soup = BeautifulSoup(reqs.text, 'html5lib')
                for title in soup.find_all('title'):
                    full_title = title.get_text()
                    if  re.search('[a-zA-Z]', full_title) or  'פייסבוק' in full_title or not full_title.isprintable():
                      continue
                    if filter_result:
                      full_title = self.clean_titles.clean_title(full_title)
                      title_list.append(full_title)
                    else:
                      title_list.append(full_title)

        except Exception as e:
            print("exception has been occur", str(e))
            # pass

        return title_list


    def get_result_from_googleNews(self, query, num_to_retrieve = 5,start_date = None,end_date = None , filter_result=False):
        title_list = []
        google_news = GoogleNews(lang='he', country='IL')
        result_list = google_news.search(query, from_=start_date,  to_=end_date)
        for result in result_list['entries']:
            full_title = result['title']

            if filter_result:
              full_title = self.clean_titles.clean_title(full_title)
              title_list.append(full_title)

            else:
              title_list.append(full_title)

        return title_list


In [ ]:
model = SentenceTransformerDocumentEmbeddings('onlplab/alephbert-base')

2022-06-13 18:32:56,171 | INFO | SentenceTransformer.py:66 | __init__ | Load pretrained SentenceTransformer: onlplab/alephbert-base


Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/545k [00:00<?, ?B/s]

2022-06-13 18:33:23,111 | WARNING | SentenceTransformer.py:796 | _load_auto_model | No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/onlplab_alephbert-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/onlplab_alephbert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical 

HeadlinesModel

In [ ]:
class HeadLineToClusterModel:
    def __init__(self, tweets_cluster, model_to_use):
        self.bert_embedding = model_to_use
        self.idx = indexer(tweets_cluster)
        self.cluster_centroid = self.calc_cluster_centroid(tweets_cluster)
        self.google_apis = GoogleModel()
        self.cos = torch.nn.CosineSimilarity()

    def calc_cluster_centroid(self, tweets_cluster):
        is_vector_initialized = False
        vector_sum = None
        try:
            for i, tweet in tweets_cluster.iterrows():
                # getting the cluster tweet as one sentence
                sentence_tokens = Sentence(tweet["text"])
                # embedding the sentence to a semantic vector
                self.bert_embedding.embed(sentence_tokens)
                # adding the semantic vector to the vectors that have been calculated so far
                vector_sum = sentence_tokens.embedding if not is_vector_initialized else vector_sum + sentence_tokens.embedding
                is_vector_initialized = True
            # averaging the cluster
            return vector_sum / len(tweets_cluster)

        except Exception as e:
            print("exception has been occur14", str(e))

    def tf_cosine_similarity(self, u, v):
        u = tf.expand_dims(u, 1)

        n = tf.reduce_sum(u * v, axis=2)
        d = tf.linalg.norm(u, axis=2) * tf.linalg.norm(v, axis=1)
        return n / d

    def calc_cosine_similarity_centroid(self, sentence):
        # this function getting a sentence and calculate the cosine similarity to the centroid
        try:
            sentence_tokens = Sentence(sentence)
            self.bert_embedding.embed(sentence_tokens)
            return 1 - cosine(self.cluster_centroid, sentence_tokens.embedding)

        except Exception as e:
            print("exception has been occur15", str(e))

    # about the headlines functions.
    # the first part of them is to calculate the terms or sentence that want to be used.
    # then each one of them use different method to get potential titles( google, top frequency terms etc..)
    # then calculating cosine to the centroid to understand how good is the title

    def get_headlines_top_frequency(self, num_of_terms):
        try:
          res = {}
          sorted_terms = sorted(self.idx.ngrams_dict, key=self.idx.ngrams_dict.get, reverse=True)[:num_of_terms]
          for term in sorted_terms:
            res[term] = self.idx.ngrams_dict[term]
          return res

        except Exception as e:
            print("exception has been occur16", str(e))


    def get_headlines_top30frequent_ngrams_similar_centroid(self, num_of_terms):
        try:
            terms_scores = {}
            res = {}
            sorted_terms = sorted(self.idx.ngrams_dict, key=self.idx.ngrams_dict.get, reverse=True)[:30]
            for term in sorted_terms:
                formlized_term = " ".join(list(term))
                terms_scores[formlized_term] = self.calc_cosine_similarity_centroid(formlized_term)

            sorted_terms = sorted(terms_scores, key=terms_scores.get, reverse=True)[:num_of_terms]
            for term in sorted_terms :
              res[term] = terms_scores[term]
            return res
        except Exception as e:
            print("exception has been occur17", str(e))


    def get_headlines_google_titles_similar_centroid(self, num_of_terms, num_of_search=3,filter_result=False, gene_ngrams=False, gene_all_kind_ngrams= False):
        try:
            # sorting the indexer according to the terms frequency
            sorted_terms = sorted(self.idx.ngrams_dict, key= self.idx.ngrams_dict.get, reverse=True)[:num_of_terms]
            titles_dict = {}
            for term in sorted_terms:
                term = " ".join(list(term))
                google_list = self.google_apis.get_num_result_for_query(term, num_of_search, filter_result=filter_result)
                print("found googles search titles", len(google_list))
                google_news_list = self.google_apis.get_result_from_googleNews(term, num_of_search, start_date = "2022-04-23",end_date = "2022-04-25", filter_result=filter_result)
                print("found news api  titles", len(google_news_list))
                combined_list  = google_list + google_news_list

                if gene_ngrams:
                  ngrams_list = self.google_apis.clean_titles.generate_ngrams(combined_list)
                  combined_list = ngrams_list

                if gene_all_kind_ngrams:
                  ngrams_list = self.google_apis.clean_titles.generate__all_kinds_of_ngrams(combined_list)
                  combined_list = ngrams_list
                for google_title in combined_list:
                    terms_similarity = self.calc_cosine_similarity_centroid(google_title)
                    titles_dict[google_title] = terms_similarity

            return titles_dict
        except Exception as e:
            print("exception has been occur18", str(e))

    def get_Ngrams_from_top_google_headlines(self, filter_headline=True, num_of_headlines=3,gene_ngrams=True, gene_all_kind_ngrams = False):
      headlines = self.get_headlines_google_titles_similar_centroid(3, filter_result=filter_headline)
      chosen_headlines = sorted(headlines, key=headlines.get, reverse=True)[:num_of_headlines]
      ngrams_list = []
      titles_dict = {}
      if gene_ngrams:
        ngrams_list = self.google_apis.clean_titles.generate_ngrams(chosen_headlines)

      if gene_all_kind_ngrams:
        ngrams_list = self.google_apis.clean_titles.generate__all_kinds_of_ngrams(chosen_headlines)

      for google_title in ngrams_list:
        terms_similarity = self.calc_cosine_similarity_centroid(google_title)
        titles_dict[google_title] = terms_similarity
      return titles_dict





In [ ]:
headline_model = HeadLineToClusterModel(df_filtered, model)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Top Frequency Terms

In [ ]:
res = headline_model.get_headlines_top_frequency(20)
print_top_n_Ngrams(res,20)

headline:  frozenset({'משפחת', 'נתניהו'}) score:  33
headline:  frozenset({'בנט', 'משפחת'}) score:  20
headline:  frozenset({'הממשלה', 'ראש'}) score:  15
headline:  frozenset({'ראש', 'ממשלה'}) score:  13
headline:  frozenset({'ההוצאות', 'נתניהו'}) score:  12
headline:  frozenset({'שרה', 'נתניהו'}) score:  12
headline:  frozenset({'שח', 'מיליון'}) score:  10
headline:  frozenset({'היא', 'מטומטמת'}) score:  9
headline:  frozenset({'הפרטי', 'ביתו'}) score:  8
headline:  frozenset({'היה', 'נתניהו'}) score:  8
headline:  frozenset({'הוצאות', 'רוה', 'מ'}) score:  7
headline:  frozenset({'הוצאות', 'המזון'}) score:  6
headline:  frozenset({'מיליון', '50'}) score:  6
headline:  frozenset({'שח', 'אלף'}) score:  6
headline:  frozenset({'הפרטי', 'בנט', 'ביתו'}) score:  6
headline:  frozenset({'למשפחת', 'נתניהו'}) score:  6
headline:  frozenset({'בנט', 'ההוצאות'}) score:  6
headline:  frozenset({'אילה', 'חסון'}) score:  6
headline:  frozenset({'שמשפחת', 'נתניהו'}) score:  6
headline:  frozenset({'ה

The Terms That Are the most close to the cluster Centroid

In [ ]:
res = headline_model.get_headlines_top30frequent_ngrams_similar_centroid(20)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print_top_n_Ngrams(res,20)

headline:  משפחת נתניהו score:  0.5399594902992249
headline:  להוצאות ומשפחתו נתניהו score:  0.538597822189331
headline:  הוצאות רוה מ score:  0.5355349183082581
headline:  למשפחת נתניהו score:  0.5304028391838074
headline:  אצל נתניהו score:  0.516991138458252
headline:  שרה נתניהו score:  0.5142388343811035
headline:  בתקופת נתניהו score:  0.5094202160835266
headline:  היא מטומטמת score:  0.5075002908706665
headline:  היה נתניהו score:  0.5073708295822144
headline:  שמשפחת נתניהו score:  0.49937164783477783
headline:  הוצאות המזון score:  0.49755436182022095
headline:  חשבון המדינה score:  0.4823685586452484
headline:  בנט משפחת score:  0.47336286306381226
headline:  ההוצאות נתניהו score:  0.4711848795413971
headline:  הוציאה נתניהו score:  0.47059696912765503
headline:  להוצאות נתניהו score:  0.4673038423061371
headline:  בנט ההוצאות score:  0.46649909019470215
headline:  שהוא רוצה score:  0.46407848596572876
headline:  שח מיליון score:  0.4603687524795532
headline:  הפרטי בנט ביתו 

Googels headlines based on centorid

In [ ]:

res = headline_model.get_headlines_google_titles_similar_centroid(3, filter_result=False)

found googles search titles 8
exception has been occur18 Could not parse your date


In [ ]:
print_top_n_Ngrams(res,30)

AttributeError: ignored

In [ ]:
res = headline_model.get_headlines_google_titles_similar_centroid(3, filter_result=True)

In [ ]:
print_top_n_Ngrams(res,30)

In [ ]:
res = headline_model.get_headlines_google_titles_similar_centroid(3, filter_result=True, gene_ngrams=True)


In [ ]:
print_top_n_Ngrams(res,30)

In [ ]:
res = headline_model.get_headlines_google_titles_similar_centroid(3, filter_result=False, gene_ngrams=True)

In [ ]:
print_top_n_Ngrams(res,30)

In [ ]:
res = headline_model.get_headlines_google_titles_similar_centroid(3, filter_result=True, gene_ngrams=False, gene_all_kind_ngrams=True)

In [ ]:
print_top_n_Ngrams(res,30)

# multilingual Bert

In [ ]:
model = SentenceTransformerDocumentEmbeddings('bert-base-multilingual-cased')
headline_model = HeadLineToClusterModel(df_filtered, model)

The Terms That Are the most close to the cluster Centroid

In [ ]:
res = headline_model.get_headlines_top30frequent_ngrams_similar_centroid(20)


In [ ]:
print_top_n_Ngrams(res,20)

Googels headlines based on centorid

In [ ]:

res = headline_model.get_headlines_google_titles_similar_centroid(3, filter_result=False)

In [ ]:
print_top_n_Ngrams(res,30)

In [ ]:
res = headline_model.get_headlines_google_titles_similar_centroid(3, filter_result=True)

In [ ]:
print_top_n_Ngrams(res,30)

In [ ]:
res = headline_model.get_headlines_google_titles_similar_centroid(3, filter_result=True, gene_ngrams=True)


In [ ]:
print_top_n_Ngrams(res,30)

In [ ]:
res = headline_model.get_headlines_google_titles_similar_centroid(3, filter_result=False, gene_ngrams=True)

In [ ]:
print_top_n_Ngrams(res,30)

In [ ]:
res = headline_model.get_headlines_google_titles_similar_centroid(3, filter_result=True, gene_ngrams=False, gene_all_kind_ngrams=True)

In [ ]:
print_top_n_Ngrams(res,30)

# multilingual Bert - Second model

In [ ]:
model = SentenceTransformerDocumentEmbeddings('paraphrase-multilingual-MiniLM-L12-v2')
headline_model = HeadLineToClusterModel(df_filtered, model)

The Terms That Are the most close to the cluster Centroid

In [ ]:
res = headline_model.get_headlines_top30frequent_ngrams_similar_centroid(20)
print_top_n_Ngrams(res,20)

Googels headlines based on centorid

In [ ]:

res = headline_model.get_headlines_google_titles_similar_centroid(3, filter_result=False)

In [ ]:
print_top_n_Ngrams(res,30)

In [ ]:
res = headline_model.get_headlines_google_titles_similar_centroid(3, filter_result=True)

In [ ]:
print_top_n_Ngrams(res,30)

In [ ]:
res = headline_model.get_headlines_google_titles_similar_centroid(3, filter_result=True, gene_ngrams=True)


In [ ]:
print_top_n_Ngrams(res,30)

In [ ]:
res = headline_model.get_headlines_google_titles_similar_centroid(3, filter_result=False, gene_ngrams=True)

In [ ]:
print_top_n_Ngrams(res,30)

In [ ]:
res = headline_model.get_headlines_google_titles_similar_centroid(3, filter_result=True, gene_ngrams=False, gene_all_kind_ngrams=True)

In [ ]:
print_top_n_Ngrams(res,30)